In [ ]:
# Import the library
import asyncio
import time
from sincpro_async_worker import run_async_task

print("✅ sincpro_async_worker loaded successfully!")

## 📝 Example 1: Basic Async Task Execution

The most common use case - executing async functions in a blocking manner:

In [ ]:
# Define a simple async task
async def fetch_data(source: str, delay: float = 1.0) -> str:
    """Simulate fetching data from an external source."""
    print(f"🔄 Fetching data from {source}...")
    await asyncio.sleep(delay)
    return f"✅ Data from {source} (took {delay}s)"

# Execute the task (blocking mode)
start_time = time.time()
result = run_async_task(fetch_data("database", 0.5))
end_time = time.time()

print(f"Result: {result}")
print(f"Total time: {end_time - start_time:.2f}s")
print(f"Result type: {type(result)}")

## 🔥 Example 2: Fire-and-Forget Execution

Execute tasks asynchronously and retrieve results later:

In [ ]:
# Start multiple tasks in fire-and-forget mode
print("🚀 Starting multiple tasks...")

future1 = run_async_task(fetch_data("API-1", 1.0), fire_and_forget=True)
future2 = run_async_task(fetch_data("API-2", 0.8), fire_and_forget=True)
future3 = run_async_task(fetch_data("API-3", 1.2), fire_and_forget=True)

print(f"Future types: {type(future1)}")
print("✅ All tasks started! Continuing with other work...")

# Simulate doing other work
time.sleep(0.2)
print("💼 Doing other work while tasks run in background...")

# Retrieve results when ready
results = []
for i, future in enumerate([future1, future2, future3], 1):
    result = future.result(timeout=5.0)  # Wait up to 5 seconds
    results.append(result)
    print(f"Task {i} completed: {result}")

print(f"\n🎉 All {len(results)} tasks completed!")

## ⚡ Example 3: Parallel Execution Within a Task

Execute multiple async operations in parallel within a single task:

In [ ]:
async def parallel_operations():
    """Execute multiple async operations in parallel."""
    print("🔄 Starting parallel operations...")
    
    # These will run in parallel within the same event loop
    tasks = [
        fetch_data("Service-A", 0.5),
        fetch_data("Service-B", 0.7),
        fetch_data("Service-C", 0.3),
        fetch_data("Service-D", 0.6)
    ]
    
    # Wait for all tasks to complete
    results = await asyncio.gather(*tasks)
    return results

# Execute the parallel task
start_time = time.time()
all_results = run_async_task(parallel_operations())
end_time = time.time()

print(f"\n📊 Results:")
for i, result in enumerate(all_results, 1):
    print(f"  {i}. {result}")

print(f"\n⏱️  Total time: {end_time - start_time:.2f}s")
print("💡 Notice: All 4 operations ran in parallel, not sequentially!")

## 🛡️ Example 4: Error Handling

Demonstrating how exceptions are properly handled:

In [ ]:
async def risky_task(will_fail: bool = False) -> str:
    """A task that might fail."""
    await asyncio.sleep(0.2)
    if will_fail:
        raise ValueError("💥 Simulated error!")
    return "✅ Task completed successfully"

# Test successful execution
print("🧪 Testing successful execution:")
try:
    result = run_async_task(risky_task(will_fail=False))
    print(f"Success: {result}")
except Exception as e:
    print(f"Unexpected error: {e}")

# Test error handling in blocking mode
print("\n🧪 Testing error handling (blocking mode):")
try:
    result = run_async_task(risky_task(will_fail=True))
    print(f"Success: {result}")
except ValueError as e:
    print(f"✅ Caught expected error: {e}")

# Test error handling in fire-and-forget mode
print("\n🧪 Testing error handling (fire-and-forget mode):")
future = run_async_task(risky_task(will_fail=True), fire_and_forget=True)
try:
    result = future.result(timeout=1.0)
    print(f"Success: {result}")
except ValueError as e:
    print(f"✅ Caught expected error from future: {e}")

## ⏰ Example 5: Timeout Handling

Demonstrating timeout functionality:

In [ ]:
async def slow_task(duration: float) -> str:
    """A task that takes a specified amount of time."""
    print(f"⏳ Starting slow task ({duration}s)...")
    await asyncio.sleep(duration)
    return f"✅ Slow task completed after {duration}s"

# Test with sufficient timeout
print("🧪 Testing with sufficient timeout:")
try:
    result = run_async_task(slow_task(0.5), timeout=2.0)
    print(f"Success: {result}")
except TimeoutError as e:
    print(f"Timeout: {e}")

# Test with insufficient timeout
print("\n🧪 Testing with insufficient timeout:")
try:
    result = run_async_task(slow_task(2.0), timeout=0.5)
    print(f"Success: {result}")
except TimeoutError as e:
    print(f"✅ Caught expected timeout: {e}")

## 🌐 Example 6: Real-World Use Case - Web API Calls

A practical example simulating multiple web API calls:

In [ ]:
async def mock_api_call(endpoint: str, delay: float = 0.5) -> dict:
    """Mock an API call with simulated network delay."""
    print(f"🌐 Calling API: {endpoint}")
    await asyncio.sleep(delay)  # Simulate network delay
    
    # Mock response data
    return {
        "endpoint": endpoint,
        "status": "success",
        "data": f"Response from {endpoint}",
        "timestamp": time.time()
    }

async def fetch_user_profile(user_id: int) -> dict:
    """Fetch complete user profile by aggregating multiple API calls."""
    print(f"👤 Fetching complete profile for user {user_id}")
    
    # These API calls will run in parallel
    profile_task = mock_api_call(f"/users/{user_id}/profile", 0.3)
    settings_task = mock_api_call(f"/users/{user_id}/settings", 0.4)
    activity_task = mock_api_call(f"/users/{user_id}/activity", 0.2)
    
    # Wait for all API calls to complete
    profile, settings, activity = await asyncio.gather(
        profile_task, settings_task, activity_task
    )
    
    return {
        "user_id": user_id,
        "profile": profile,
        "settings": settings,
        "activity": activity,
        "aggregated_at": time.time()
    }

# Execute the complex operation
print("🚀 Fetching user profile with multiple API calls...")
start_time = time.time()

user_data = run_async_task(fetch_user_profile(123))

end_time = time.time()

print(f"\n📋 User Profile Retrieved:")
print(f"  User ID: {user_data['user_id']}")
print(f"  Profile API: {user_data['profile']['status']}")
print(f"  Settings API: {user_data['settings']['status']}")
print(f"  Activity API: {user_data['activity']['status']}")
print(f"\n⏱️  Total time: {end_time - start_time:.2f}s")
print("💡 All 3 API calls executed in parallel!")

## 🎯 Example 7: Comparison - Serial vs Parallel Execution

Demonstrating the performance benefits of parallel execution:

In [ ]:
async def serial_execution():
    """Execute tasks one after another (serial)."""
    results = []
    for i in range(3):
        result = await fetch_data(f"Serial-{i+1}", 0.4)
        results.append(result)
    return results

async def parallel_execution():
    """Execute tasks in parallel."""
    tasks = [
        fetch_data(f"Parallel-{i+1}", 0.4) 
        for i in range(3)
    ]
    return await asyncio.gather(*tasks)

# Test serial execution
print("🐌 Testing serial execution:")
start_time = time.time()
serial_results = run_async_task(serial_execution())
serial_time = time.time() - start_time
print(f"Serial execution time: {serial_time:.2f}s")

# Test parallel execution
print("\n⚡ Testing parallel execution:")
start_time = time.time()
parallel_results = run_async_task(parallel_execution())
parallel_time = time.time() - start_time
print(f"Parallel execution time: {parallel_time:.2f}s")

# Compare results
print(f"\n📊 Performance Comparison:")
print(f"  Serial:   {serial_time:.2f}s")
print(f"  Parallel: {parallel_time:.2f}s")
print(f"  Speedup:  {serial_time/parallel_time:.1f}x faster")

## 📚 Summary

You've learned how to use `sincpro_async_worker` for:

1. **Basic async task execution** - Simple blocking execution
2. **Fire-and-forget tasks** - Non-blocking execution with futures
3. **Parallel operations** - Multiple async operations within a single task
4. **Error handling** - Proper exception propagation
5. **Timeout management** - Controlling maximum execution time
6. **Real-world scenarios** - Web API calls and data aggregation
7. **Performance optimization** - Parallel vs serial execution

## 🎉 Key Takeaways

- Use `fire_and_forget=False` (default) for blocking execution
- Use `fire_and_forget=True` for non-blocking execution
- Combine multiple async operations with `asyncio.gather()` for parallelism
- Always handle exceptions appropriately
- Use timeouts for operations that might hang
- The library works consistently across all Python environments

Happy async programming! 🚀